[![Roboflow Notebooks](https://media.roboflow.com/notebooks/template/bannertest2-2.png?ik-sdk-version=javascript-1.4.3&updatedAt=1672932710194)](https://github.com/roboflow/notebooks)

# How to Track Objects with RF-DETR and SORT Tracker

SORT is a classic online, tracking-by-detection method that predicts object motion with a Kalman filter and matches predicted tracks to detections using the Hungarian algorithm based on Intersection over Union (IoU). The tracker uses only geometric cues from bounding boxes, without appearance features, so it runs extremely fast and scales to hundreds of frames per second on typical hardware. Detections from a strong CNN detector feed SORT, which updates each track’s state via a constant velocity motion model and prunes stale tracks. Because SORT lacks explicit re-identification or appearance cues, it can suffer identity switches and fragmented tracks under long occlusions or heavy crowding.

## Setup

### Check GPU availability

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Runtime` -> `Change runtime type` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [ ]:
!nvidia-smi

### Install dependencies

You may see dependency conflict warnings in Google Colab. This is expected for the preinstalled Google Colab environment and does not affect functionality.

In [ ]:
!pip install -q inference-gpu trackers==2.2.0

### Download example data

Downloads example videos for testing. You can use these or replace them with your own images.

In [ ]:
!wget -q https://storage.googleapis.com/com-roboflow-marketing/supervision/video-examples/bikes-1280x720-1.mp4
!wget -q https://storage.googleapis.com/com-roboflow-marketing/supervision/video-examples/bikes-1280x720-2.mp4
!wget -q https://storage.googleapis.com/com-roboflow-marketing/supervision/video-examples/skiers-1280x720-5.mp4

## Track from CLI

The `trackers` library provides a convenient command-line interface (CLI) for running object tracking on videos. You can quickly track objects without writing any Python code. Below is a simple example showing how to track objects using the SORT tracker.

For more details on CLI options and configurations, visit the [Track from CLI documentation](https://trackers.roboflow.com/develop/learn/track/).

In [ ]:
SOURCE_VIDEO_PATH = "/content/bikes-1280x720-1.mp4"
TARGET_VIDEO_PATH = "/content/bikes-1280x720-1-result.mp4"

!trackers track \
    --source {SOURCE_VIDEO_PATH} \
    --output {TARGET_VIDEO_PATH} \
    --model rfdetr-medium \
    --tracker sort \
    --show-trajectories

In [ ]:
TARGET_VIDEO_COMPRESSED_PATH = "/content/bikes-1280x720-1-result-compressed.mp4"

!ffmpeg -y -loglevel error -i {TARGET_VIDEO_PATH} -vcodec libx264 -crf 28 {TARGET_VIDEO_COMPRESSED_PATH}

In [ ]:
from IPython.display import Video

Video(TARGET_VIDEO_COMPRESSED_PATH, embed=True, width=1080)

## Track from Python

### Initiate detector and tracker

In [ ]:
from inference import get_model
from trackers import SORTTracker

model = get_model("rfdetr-medium")
tracker = SORTTracker()

### Configure annotators

In [ ]:
import supervision as sv

color = sv.ColorPalette.from_hex([
    "#ffff00", "#ff9b00", "#ff8080", "#ff66b2", "#ff66ff", "#b266ff",
    "#9999ff", "#3399ff", "#66ffff", "#33ff99", "#66ff66", "#99ff00"
])

box_annotator = sv.BoxAnnotator(
    color=color,
    color_lookup=sv.ColorLookup.TRACK)

label_annotator = sv.LabelAnnotator(
    color=color,
    color_lookup=sv.ColorLookup.TRACK,
    text_color=sv.Color.BLACK,
    text_scale=0.8)

trace_annotator = sv.TraceAnnotator(
    color=color,
    color_lookup=sv.ColorLookup.TRACK,
    thickness=2,
    trace_length=100)

### Run detection + tracking

In [ ]:
CONFIDENCE_THRESHOLD = 0.2
NMS_THRESHOLD = 0.3

SOURCE_VIDEO_PATH = "/content/bikes-1280x720-2.mp4"
TARGET_VIDEO_PATH = "/content/bikes-1280x720-2-result.mp4"

def callback(frame, i):
    result = model.infer(frame, confidence=CONFIDENCE_THRESHOLD)[0]
    detections = sv.Detections.from_inference(result).with_nms(threshold=NMS_THRESHOLD)
    detections = tracker.update(detections)

    annotated_image = frame.copy()
    annotated_image = box_annotator.annotate(annotated_image, detections)
    annotated_image = trace_annotator.annotate(annotated_image, detections)
    annotated_image = label_annotator.annotate(annotated_image, detections, detections.tracker_id)

    return annotated_image

tracker.reset()

sv.process_video(
    source_path=SOURCE_VIDEO_PATH,
    target_path=TARGET_VIDEO_PATH,
    callback=callback,
    show_progress=True,
)

### Display result

In [ ]:
TARGET_VIDEO_COMPRESSED_PATH = "/content/bikes-1280x720-2-result-compressed.mp4"

!ffmpeg -y -loglevel error -i {TARGET_VIDEO_PATH} -vcodec libx264 -crf 28 {TARGET_VIDEO_COMPRESSED_PATH}

In [ ]:
from IPython.display import Video

Video(TARGET_VIDEO_COMPRESSED_PATH, embed=True, width=1080)

### Camera motion compensation

When tracking objects with a moving camera, trajectories can appear unstable as the camera pans or tilts. Camera motion compensation uses optical flow to estimate camera movement and stabilize trajectory visualization. This is especially useful for footage from drones, handheld cameras, or any scenario where the camera itself is in motion.

In [ ]:
from inference import get_model
from trackers import SORTTracker, MotionEstimator, MotionAwareTraceAnnotator

PERSON_CLASS_ID = 0

model = get_model("rfdetr-large")
tracker = SORTTracker(minimum_consecutive_frames=3)
motion_estimator = MotionEstimator(
    max_points=500,
    min_distance=10,
    quality_level=0.001,
    ransac_reproj_threshold=1.0,
)

color = sv.ColorPalette.from_hex([
    "#ffff00", "#ff9b00", "#ff8080", "#ff66b2", "#ff66ff", "#b266ff",
    "#9999ff", "#3399ff", "#66ffff", "#33ff99", "#66ff66", "#99ff00"
])

box_annotator = sv.BoxAnnotator(
    color=color,
    color_lookup=sv.ColorLookup.TRACK)

label_annotator = sv.LabelAnnotator(
    color=color,
    color_lookup=sv.ColorLookup.TRACK,
    text_color=sv.Color.BLACK,
    text_scale=0.8)

motion_aware_trace_annotator = MotionAwareTraceAnnotator(
    color=color,
    color_lookup=sv.ColorLookup.TRACK,
    thickness=2,
    trace_length=100)

In [ ]:
CONFIDENCE_THRESHOLD = 0.2
NMS_THRESHOLD = 0.3

SOURCE_VIDEO_PATH = "/content/skiers-1280x720-5.mp4"
TARGET_VIDEO_PATH = "/content/skiers-1280x720-5-result.mp4"

def callback(frame, i):
    coord_transform = motion_estimator.update(frame)

    result = model.infer(frame, confidence=CONFIDENCE_THRESHOLD)[0]
    detections = sv.Detections.from_inference(result).with_nms(threshold=NMS_THRESHOLD)
    detections = detections[detections.class_id == PERSON_CLASS_ID]
    detections = tracker.update(detections)

    annotated_image = frame.copy()
    annotated_image = box_annotator.annotate(annotated_image, detections)
    annotated_image = motion_aware_trace_annotator.annotate(
        annotated_image, detections, coord_transform=coord_transform)
    annotated_image = label_annotator.annotate(annotated_image, detections, detections.tracker_id)

    return annotated_image

tracker.reset()
motion_estimator.reset()
motion_aware_trace_annotator.reset()

sv.process_video(
    source_path=SOURCE_VIDEO_PATH,
    target_path=TARGET_VIDEO_PATH,
    callback=callback,
    show_progress=True,
)

In [ ]:
TARGET_VIDEO_COMPRESSED_PATH = "/content/skiers-1280x720-5-result-compressed.mp4"

!ffmpeg -y -loglevel error -i {TARGET_VIDEO_PATH} -vcodec libx264 -crf 28 {TARGET_VIDEO_COMPRESSED_PATH}

In [ ]:
from IPython.display import Video

Video(TARGET_VIDEO_COMPRESSED_PATH, embed=True, width=1080)

You just combined SORT tracker with RF-DETR detector. Nice work!

Trackers makes it easy to mix and match top-performing multi-object tracking algorithms with your favorite detection backends, including Inference, Ultralytics, and Transformers.

Ready to go deeper. Explore more tracking algorithms and integrations in the trackers [Documentation](https://roboflow.github.io/trackers/) or dive into the code on [GitHub](https://github.com/roboflow/trackers).

Got feedback or ideas? Open an issue on [GitHub Issues](https://github.com/roboflow/trackers/issues).